In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


import matplotlib.pyplot as plt

# from darts import TimeSeries
# from darts.dataprocessing.transformers import Scaler
# from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
# from darts.metrics import mape
# from darts.utils.statistics import check_seasonality, plot_acf
# from darts.datasets import AirPassengersDataset, SunspotsDataset
# from darts.utils.timeseries_generation import datetime_attribute_timeseries

# import warnings

# warnings.filterwarnings("ignore")
import logging

import tensorflow
from tensorflow import expand_dims



from datetime import datetime

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from xgboost import XGBRegressor
from matplotlib import pyplot

# LSTM 2017-2018


In [5]:
df = pd.read_csv('../raw_data/final_df.csv', index_col=0)


In [6]:
df.index = pd.to_datetime(df.index)
df

,Brent_Crude_GBP,WTI_Crude_GBP,GBP Curncy,GBPEUR Curncy,UKX Index,SPX Index,SX5E Index,Natural_Gas,CPI,RPI,GBP_IRS_2y,GBP_IRS_10y,Nationwide_HPI,RPI YOY
Date,,,,,,,,,,,,,,
2009-01-01,30.903838,33.424333,1.479027,1.099905,4308.390000,596.071497,2135.982324,12.472805,84.9,210.1,2.122545,3.757500,300.2,0.142993
2009-02-01,29.393166,33.617553,1.492643,1.120824,4232.114762,570.410394,2020.716905,9.467866,85.6,211.4,2.129905,3.798333,294.7,0.000000
2009-03-01,31.733352,36.826152,1.494241,1.097877,4107.840909,566.393975,2029.160819,8.064295,85.8,211.3,2.142045,3.872500,301.1,-0.377181
2009-04-01,33.990786,36.012519,1.511936,1.125327,4259.890500,588.042125,2119.383029,7.762397,86.0,211.5,2.143250,3.935500,302.9,-1.168224
2009-05-01,37.524515,38.801542,1.561252,1.131686,4434.719500,582.718727,2164.120107,7.757394,86.4,212.8,2.031900,3.917500,307.2,-1.069270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-01,89.748267,85.128666,1.247484,1.178074,7472.767778,3274.256696,3179.554466,81.073170,120.8,337.1,2.538121,2.240542,538.4,11.659490
2022-06-01,95.251442,87.836565,1.233611,1.167758,7238.188421,3172.340166,3058.969505,96.746393,121.8,340.0,2.933563,2.626558,541.8,11.842105
2022-07-01,85.363871,80.415504,1.231479,1.184589,7303.865263,3298.689210,3082.959231,128.688654,122.5,343.2,2.700779,2.302258,541.0,12.340426


In [7]:
#rpi_lag = pd.read_csv('RPI_lagged.csv')


In [8]:
#rpi_lag

In [9]:
# rpi_lag = rpi_lag.rename({'Date CO1 Comdty': 'Date'}, axis=1)
# rpi_lag = rpi_lag.set_index('Date')


In [10]:
#rpi_lag.index = pd.to_datetime(rpi_lag.index)

In [11]:
#rpi_lag

In [12]:
#new_df = df.join(rpi_lag)

In [13]:
#new_df

In [14]:
# X = new_df.drop(columns = ['RPI', 'CPI', 'RPI YOY'])
# y = new_df['RPI']

X = df.drop(columns = ['RPI', 'CPI', 'RPI YOY'])
y = df['RPI']

In [15]:
X[72:96]

,Brent_Crude_GBP,WTI_Crude_GBP,GBP Curncy,GBPEUR Curncy,UKX Index,SPX Index,SX5E Index,Natural_Gas,GBP_IRS_2y,GBP_IRS_10y,Nationwide_HPI
Date,,,,,,,,,,,
2015-01-01,32.818125,31.737054,1.516177,1.336264,6641.765238,1339.503656,2495.664852,13.099203,0.858274,1.684729,375.9
2015-02-01,38.343255,32.759207,1.533386,1.362371,6746.840476,1356.232800,2536.801725,13.938965,0.952887,1.856450,375.0
2015-03-01,37.866261,31.635113,1.503677,1.377568,6791.140000,1380.431518,2615.694574,13.821672,0.947216,1.783045,377.9
2015-04-01,40.409377,35.572047,1.512909,1.386036,6886.650952,1381.813784,2620.084413,13.690119,0.923663,1.811432,385.1
2015-05-01,42.505052,36.186768,1.543548,1.384014,6864.994500,1360.216539,2568.111738,13.123988,1.009908,1.971630,389.3
2015-06-01,41.067582,35.885473,1.552386,1.385524,6687.803500,1340.919813,2507.691659,13.023503,1.061182,2.089545,389.1
2015-07-01,36.726002,31.989864,1.545617,1.402913,6625.526087,1353.218881,2518.626151,13.169073,1.119989,2.051104,390.2
2015-08-01,31.196110,29.152671,1.545248,1.376957,6402.255000,1311.027051,2434.955959,12.478583,1.085050,1.958367,389.6
2015-09-01,31.774855,30.721787,1.527609,1.364657,6280.524348,1302.101893,2375.712634,12.595702,1.004023,1.908904,390.2


In [16]:
X_train, y_train = X[:72], y[:72]
X_val, y_val = X[72:96], y[72:96]
X_test, y_test = X[96:108], y[96:108]
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((72, 11), (72,), (24, 11), (24,), (12, 11), (12,))

In [21]:

y_train_x = expand_dims(y_train, -1)
y_train_x = expand_dims(y_train_x, -1)

y_train_x.shape

TensorShape([72, 1, 1])

In [22]:
y_val_x = expand_dims(y_val, -1)
y_val_x = expand_dims(y_val_x, -1)

In [23]:
y_val_x.shape

TensorShape([24, 1, 1])

In [24]:
X_test_x = expand_dims(X_test, -2)

In [25]:
X_test_x.shape

TensorShape([12, 1, 11])

In [26]:
X_train_x = expand_dims(X_train, -2)
X_train_x.shape

TensorShape([72, 1, 11])

In [27]:
X_val_x = expand_dims(X_val, -2)
X_val_x.shape

TensorShape([24, 1, 11])

In [31]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

lr_scheduler = ExponentialDecay(initial_learning_rate=0.001, decay_steps= 10_000, decay_rate=0.7)
adam = Adam(learning_rate=lr_scheduler)
es = EarlyStopping(patience=50, restore_best_weights=True)

In [32]:
from tensorflow.keras.layers import Normalization

normalizer = Normalization()
normalizer.adapt(X_train_x)

model1 = Sequential()
model1.add(normalizer)

model1.add(LSTM(units = 100, activation='tanh', return_sequences = True))
model1.add(LSTM(units = 50 , activation='tanh', return_sequences = False))

model1.add(Flatten())

model1.add(Dense(100,activation =  'relu'))
model1.add(Dense(50,activation =  'relu'))
model1.add(Dense(25,activation =  'relu'))

model1.add(Dense(1, activation = 'linear'))

model1.summary()

ImportError: cannot import name 'Normalization' from 'tensorflow.keras.layers' (/Users/gwh/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/tensorflow/keras/layers/__init__.py)

In [29]:
mape = MeanAbsolutePercentageError()

NameError: name 'MeanAbsolutePercentageError' is not defined

In [ ]:
model1.compile(loss= mape, optimizer= adam, metrics=['mape'])

: 

In [ ]:
model1.fit(X_train_x, y_train,batch_size = 32, validation_data=(X_val_x, y_val), epochs=10000, callbacks = (es))

: 

In [ ]:
model1.predict(X_test_x)

: 

In [ ]:
test_predictions = list(model1.predict(X_test_x))
test_results = pd.DataFrame(columns = ['test_predictions', 'test_actual'])

#train_predictions

test_results['date'] = list(y_test.index)
test_results['test_predictions'] = [x[0] for x in test_predictions]
test_results['test_actual'] = list(y_test)


: 

In [ ]:
test_results

: 

In [ ]:
test_results.set_index('date', inplace=True)

: 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(y)
plt.plot(test_results['test_predictions'], label='prediction')
plt.plot(test_results['test_actual'], label='actual')
plt.title('2017-18 Inflation Prediciton')
plt.legend()
#plt.savefig('LSTM_new')



: 

# LSTM 2018-2019

In [ ]:
X_train1, y_train1 = X[:84], y[:84]
X_val1, y_val1 = X[84:108], y[84:108]
X_test1, y_test1 = X[108:120], y[108:120]
X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape

: 

In [ ]:
y_train1_x = expand_dims(y_train1, -1)
y_train1_x = expand_dims(y_train1_x, -1)

y_train1_x.shape

: 

In [ ]:
y_val1_x = expand_dims(y_val1, -1)
y_val1_x = expand_dims(y_val1_x, -1)

: 

In [ ]:
X_test1_x = expand_dims(X_test1, -2)

: 

In [ ]:
X_train1_x = expand_dims(X_train1, -2)
X_train1_x.shape

: 

In [ ]:
X_val1_x = expand_dims(X_val1, -2)
X_val1_x.shape

: 

In [ ]:
lr_scheduler = ExponentialDecay(initial_learning_rate=0.001, decay_steps= 10_000, decay_rate=0.7)
adam = Adam(learning_rate=lr_scheduler)
es = EarlyStopping(patience=50, restore_best_weights=True)

: 

In [ ]:
normalizer = Normalization()
normalizer.adapt(X_train1_x)

model2 = Sequential()
model2.add(normalizer)

model2.add(LSTM(units = 100, activation='tanh', return_sequences = True))
model2.add(LSTM(units = 50 , activation='tanh', return_sequences = False))

model2.add(Flatten())

model2.add(Dense(100,activation =  'relu'))
model2.add(Dense(50,activation =  'relu'))
model2.add(Dense(25,activation =  'relu'))

model2.add(Dense(1, activation = 'linear'))

model2.summary()

: 

In [ ]:
model2.compile(loss= mape, optimizer= adam, metrics=['mape'])

: 

In [ ]:
model2.fit(X_train1_x, y_train1,batch_size = 32, validation_data=(X_val1_x, y_val1), epochs=10000, callbacks = (es))

: 

In [ ]:
test_prediction1 = list(model2.predict(X_test1_x))

: 

In [ ]:
test_results1 = pd.DataFrame(columns = ['test_predictions', 'test_actual'])

#train_predictions

test_results1['date'] = list(y_test1.index)
test_results1['test_predictions'] = [x[0] for x in test_prediction1]
test_results1['test_actual'] = list(y_test1)

: 

In [ ]:
test_results1

: 

In [ ]:
test_results1.set_index('date', inplace=True)

: 

In [ ]:
plt.plot(y)
plt.plot(test_results1['test_predictions'], label='prediction')
plt.plot(test_results1['test_actual'], label='actual')
plt.title('2018-19 Inflation Prediciton')
plt.legend()

: 

# LSTM 2019-2020

In [ ]:
X_train2, y_train2 = X[:96], y[:96]
X_val2, y_val2 = X[96:120], y[96:120]
X_test2, y_test2 = X[120:132], y[120:132]
X_train2.shape, y_train2.shape, X_val2.shape, y_val2.shape, X_test2.shape, y_test2.shape

: 

In [ ]:
y_train2_x = expand_dims(y_train2, -1)
y_train2_x = expand_dims(y_train2_x, -1)

y_train2_x.shape

: 

In [ ]:
y_val2_x = expand_dims(y_val2, -1)
y_val2_x = expand_dims(y_val2_x, -1)

: 

In [ ]:
X_test2_x = expand_dims(X_test2, -2)

: 

In [ ]:
X_train2_x = expand_dims(X_train2, -2)
X_train2_x.shape

: 

In [ ]:
X_val2_x = expand_dims(X_val2, -2)
X_val2_x.shape

: 

In [ ]:
normalizer = Normalization()
normalizer.adapt(X_train2_x)

model3 = Sequential()
model3.add(normalizer)

model3.add(LSTM(units = 100, activation='tanh', return_sequences = True))
model3.add(LSTM(units = 50 , activation='tanh', return_sequences = False))

model3.add(Flatten())

model3.add(Dense(100,activation =  'relu'))
model3.add(Dense(50,activation =  'relu'))
model3.add(Dense(25,activation =  'relu'))

model3.add(Dense(1, activation = 'linear'))

model3.summary()

: 

In [ ]:
model3.compile(loss= mape, optimizer= adam, metrics=['mape'])

: 

In [ ]:
model3.fit(X_train2_x, y_train2,batch_size = 32, validation_data=(X_val2_x, y_val2), epochs=10000, callbacks = (es))

: 

In [ ]:
test_prediction2 = list(model3.predict(X_test2_x))


: 

In [ ]:
test_results2 = pd.DataFrame(columns = ['test_predictions', 'test_actual'])

#train_predictions

test_results2['date'] = list(y_test2.index)
test_results2['test_predictions'] = [x[0] for x in test_prediction2]
test_results2['test_actual'] = list(y[120:132])

: 

In [ ]:
test_results2.set_index('date', inplace=True)

: 

In [ ]:
plt.plot(y)
plt.plot(test_results2['test_predictions'], label='prediction')
plt.plot(test_results2['test_actual'], label='actual')
plt.title('2019-20 Inflation Prediciton')
plt.legend()
#plt.savefig('LSTM_2019-20')

: 

# LSTM 2020-2021

In [ ]:
X[132:144]

: 

In [ ]:
X_train3, y_train3 = X[:108], y[:108]
X_val3, y_val3 = X[108:132], y[108:132]
X_test3, y_test3 = X[132:144], y[132:144]
X_train3.shape, y_train3.shape, X_val3.shape, y_val3.shape, X_test3.shape, y_test3.shape

: 

In [ ]:
y_train3_x = expand_dims(y_train3, -1)
y_train3_x = expand_dims(y_train3_x, -1)

y_train3_x.shape

: 

In [ ]:
y_val3_x = expand_dims(y_val3, -1)
y_val3_x = expand_dims(y_val3_x, -1)

: 

In [ ]:
X_test3_x = expand_dims(X_test3, -2)

: 

In [ ]:
X_train3_x = expand_dims(X_train3, -2)
X_train3_x.shape

: 

In [ ]:
X_val3_x = expand_dims(X_val3, -2)
X_val3_x.shape

: 

In [ ]:
normalizer = Normalization()
normalizer.adapt(X_train3_x)

model4 = Sequential()
model4.add(normalizer)

model4.add(LSTM(units = 100, activation='tanh', return_sequences = True))
model4.add(LSTM(units = 50 , activation='tanh', return_sequences = False))

model4.add(Flatten())

model4.add(Dense(100,activation =  'relu'))
model4.add(Dense(50,activation =  'relu'))
model4.add(Dense(25,activation =  'relu'))

model4.add(Dense(1, activation = 'linear'))

model4.summary()

: 

In [ ]:
model4.compile(loss= mape, optimizer= adam, metrics=['mape'])

: 

In [ ]:
model4.fit(X_train3_x, y_train3,batch_size = 32, validation_data=(X_val3_x, y_val3), epochs=10000, callbacks = (es))

: 

In [ ]:
test_prediction3 = list(model4.predict(X_test3_x))


: 

In [ ]:
test_results3 = pd.DataFrame(columns = ['test_predictions', 'test_actual'])

#train_predictions

test_results3['date'] = list(y_test3.index)
test_results3['test_predictions'] = [x[0] for x in test_prediction3]
test_results3['test_actual'] = list(y_test3)

: 

In [ ]:
test_results3.set_index('date', inplace=True)

: 

In [ ]:
plt.plot(y)
plt.plot(test_results3['test_predictions'], label='prediction')
plt.plot(test_results3['test_actual'], label='actual')
plt.title('2020-21 Inflation Prediciton')
plt.legend()
plt.savefig('LSTM_2020-21')

: 

# LSTM 2021-2022

In [ ]:
X[:165]

: 

In [ ]:
X_train4, y_train4 = X[:132], y[:132]
X_val4, y_val4 = X[132:156], y[132:156]
X_test4, y_test4 = X[156:165], y[156:165]
X_train4.shape, y_train4.shape, X_val4.shape, y_val4.shape, X_test4.shape, y_test4.shape

: 

In [ ]:
y_train4_x = expand_dims(y_train4, -1)
y_train4_x = expand_dims(y_train4_x, -1)

y_train4_x.shape

: 

In [ ]:
y_val4_x = expand_dims(y_val4, -1)
y_val4_x = expand_dims(y_val4_x, -1)

: 

In [ ]:
X_test4_x = expand_dims(X_test4, -2)

: 

In [ ]:
X_train4_x = expand_dims(X_train4, -2)
X_train4_x.shape

: 

In [ ]:
X_val4_x = expand_dims(X_val4, -2)
X_val4_x.shape

: 

In [ ]:
normalizer = Normalization()
normalizer.adapt(X_train4_x)

model5 = Sequential()
model5.add(normalizer)

model5.add(LSTM(units = 100, activation='tanh', return_sequences = True))
model5.add(LSTM(units = 50 , activation='tanh', return_sequences = False))

model5.add(Flatten())

model5.add(Dense(100,activation =  'relu'))
model5.add(Dense(50,activation =  'relu'))
model5.add(Dense(25,activation =  'relu'))

model5.add(Dense(1, activation = 'linear'))

model5.summary()

: 

In [ ]:
model5.compile(loss= mape, optimizer= adam, metrics=['mape'])

: 

In [ ]:
model5.fit(X_train4_x, y_train4,batch_size = 32, validation_data=(X_val4_x, y_val4), epochs=10000, callbacks = (es))

: 

In [ ]:
test_prediction4 = list(model5.predict(X_test4_x))
test_prediction4


: 

In [ ]:
test_results4 = pd.DataFrame(columns = ['test_predictions', 'test_actual'])

#train_predictions

test_results4['date'] = list(y_test4.index)
test_results4['test_predictions'] = [x[0] for x in test_prediction4]
test_results4['test_actual'] = list(y_test4)

: 

In [ ]:
test_results4.set_index('date', inplace=True)

: 

In [ ]:
plt.plot(y)
plt.plot(test_results4['test_predictions'], label='prediction')
plt.plot(test_results4['test_actual'], label='actual')
plt.title('2021-22 Inflation Prediciton')
plt.legend()

: 

# LSTM 2022-

In [ ]:
X[159:165]

: 

In [ ]:
X_train5, y_train5 = X[:156], y[:156]
X_val5, y_val5 = X[156:163], y[156:163]
X_test5, y_test5 = X[163:164], y[163:164]
X_train5.shape, y_train5.shape, X_val5.shape, y_val5.shape, X_test5.shape, y_test5.shape

: 

In [ ]:
y_train5_x = expand_dims(y_train5, -1)
y_train5_x = expand_dims(y_train5_x, -1)

y_train5_x.shape

: 

In [ ]:
y_val5_x = expand_dims(y_val5, -1)
y_val5_x = expand_dims(y_val5_x, -1)

: 

In [ ]:
X_test5_x = expand_dims(X_test5, -2)

: 

In [ ]:
X_train5_x = expand_dims(X_train5, -2)
X_train5_x.shape

: 

In [ ]:
X_val5_x = expand_dims(X_val5, -2)
X_val5_x.shape

: 

In [ ]:
normalizer = Normalization()
normalizer.adapt(X_train5_x)

model6 = Sequential()
model6.add(normalizer)

model6.add(LSTM(units = 110, activation='tanh', return_sequences = True))
model6.add(LSTM(units = 50 , activation='tanh', return_sequences = False))

model6.add(Flatten())

model6.add(Dense(100,activation =  'relu'))
model6.add(Dense(50,activation =  'relu'))
model6.add(Dense(25,activation =  'relu'))
model6.add(Dense(25,activation =  'relu'))
model6.add(Dense(12,activation =  'relu'))




model6.add(Dense(1, activation = 'linear'))

model6.summary()

: 

In [ ]:
model6.compile(loss= mape, optimizer= adam, metrics=['mape'])

: 

In [ ]:
model6.fit(X_train5_x, y_train5, batch_size = 16, validation_data=(X_val5_x, y_val5), epochs=10000, callbacks = (es))

: 

In [ ]:
test_prediction5 = list(model6.predict(X_test5_x))


: 

In [ ]:
test_prediction5

: 

In [ ]:
y_test5

: 

In [ ]:
# res = []
# for x in range(1):
#     val = pd.to_datetime(pd.DataFrame(y).iloc[-x,:0]).name
#     res.append(val)

: 

In [ ]:
# res = res[1:]

: 

In [ ]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

: 

In [ ]:
# list(y_test5[:-1])

: 

In [ ]:
# future_vals = []
# for x in res:
#     future_date = x + relativedelta(months=1)
#     future_vals.append(future_date)

: 

In [ ]:
# future_vals

: 

In [ ]:
test_results5 = pd.DataFrame(columns = ['test_predictions', 'test_actual'])

#train_predictions

test_results5['date'] = y_test5.index
test_results5['test_predictions'] = [x[0] for x in test_prediction5]
print(test_results5['date'])

: 

In [ ]:
test_results5.set_index('date', inplace=True)

: 

: 

In [ ]:
plt.plot(y, label='RPI')
plt.scatter([y_test5.index], test_results5['test_predictions'], label='prediction')
plt.title('2022-2023 Inflation Prediciton')
plt.legend()
#plt.savefig('LSTM_Pred_5th_run')

: 

In [ ]:
!pwd


: 

In [ ]:
model6.save('../data/time_series_final_pred.h5')

: 

In [ ]:
fig, ax = plt.subplots()
ax.plot(y, label = 'RPI')
ax.scatter([y_test5.index], test_results5['test_predictions'], label='prediction')
ax.set_title('Inflation Prediciton')
ax.legend()
fig.show()

: 

: 